In [1]:
import pandas as pd
import numpy as np
from sodapy import Socrata
import collections;
import re;

In [ ]:
client = Socrata("data.sfgov.org", None)
# results = client.get("cuks-n6tp", limit = 2191368)
data = client.get("cuks-n6tp", limit = 3000000)

In [ ]:
data_df = pd.DataFrame.from_records(data)

In [ ]:
print(type(data_df))
print(data_df.shape)

In [ ]:
print(data_df[:4])

In [6]:
data_df.sort_values(by=['date'])
data_df.

In [7]:
print(data_df.shape)

(2193544, 13)


In [8]:
for col in data_df.columns:
    print(col)

address
category
date
dayofweek
descript
incidntnum
location
pddistrict
pdid
resolution
time
x
y


In [9]:
def convert_date_to_day(dt):
   result = re.findall(r'\d{4}-(\d{2})-(\d{2})T00:00:00.000',dt)
   return result[0][1]
   
def convert_date_to_month(dt):
   result = re.findall(r'\d{4}-(\d{2})-(\d{2})T00:00:00.000',dt)
   return result[0][0]

def convert_time_to_hour(tm):
   result = re.findall(r'(\d{2}):\d{2}',tm)
   return result[0]

In [11]:
data_df = data_df.dropna(how='any',axis=0)
print(data_df[:0])

Empty DataFrame
Columns: [address, category, date, dayofweek, descript, incidntnum, location, pddistrict, pdid, resolution, time, x, y]
Index: []


In [12]:
data_df['day'] = data_df.date.apply(lambda x: convert_date_to_day(x))
data_df['month'] = data_df.date.apply(lambda x: convert_date_to_month(x))
data_df['hour'] = data_df.time.apply(lambda x: convert_time_to_hour(x))
data_df = data_df.sort_values(by=['date','time'])


In [14]:
df = data_df.drop(['incidntnum','pdid','resolution','x','y', 'date', 'time', 'descript'], axis =1)
# df = df.drop(df.columns[[0]], axis =1)

for col in df.columns:
    print(col)

address
category
dayofweek
descript
location
pddistrict
day
month
hour


In [15]:
# df.sort_values(by=['date'])
df['descript'] = df.descript.apply(lambda x: x.lower())
df['category'] = df.category.apply(lambda x: x.lower())
df['dayofweek'] = df.dayofweek.apply(lambda x: x.lower())
df['address'] = df.address.apply(lambda x: x.lower())
df['pddistrict'] = df.pddistrict.apply(lambda x: x.lower())

In [16]:
print(df.shape)

(2193543, 9)


In [17]:
series = df.groupby('pddistrict').apply(list)
dictionary = {}


In [19]:
city_target = df.get('category').tolist()
unique_target=set(city_target)
print(len(unique_target))
print(len(city_target))

39
2193543


In [20]:
#For city level
# df = df.drop(['pddistrict', 'category'], axis =1)

In [22]:
for s in series.keys():
    print(str(s))
    mask = df['pddistrict'] == str(s)
#     print(df[mask])
    dictionary[str(s)] = df[mask]

bayview
central
ingleside
mission
northern
park
richmond
southern
taraval
tenderloin


In [23]:
print(type(dictionary['central']))
print(dictionary.keys())

<class 'pandas.core.frame.DataFrame'>
['central', 'northern', 'ingleside', 'park', 'mission', 'tenderloin', 'richmond', 'taraval', 'bayview', 'southern']


In [24]:
def generateTrainData_Target(district):
    target = (dictionary.get(district))['category']
#     tempdf = (dictionary.get(district)).drop(['pddistrict', 'category'], axis =1).sort_values(by=['date'])
    tempdf = (dictionary.get(district)).drop(['pddistrict', 'category'], axis =1)
    return tempdf, target
#     dictionary

In [25]:
district_df, district_target = generateTrainData_Target('bayview')
print(len(district_target))
print(district_df.shape)

219169
(219169, 7)


In [27]:
print(district_df[:10])

                            address  dayofweek  \
40972             3rd st / cargo wy  wednesday   
41925        100 block of dakota st  wednesday   
203268   2000 block of mc kinnon av  wednesday   
245057      1500 block of hudson av  wednesday   
324590      500 block of holyoke st  wednesday   
386011        0 block of osceola ln  wednesday   
415338        0 block of osceola ln  wednesday   
468458      1300 block of silver av  wednesday   
549495   100 block of bridgeview dr  wednesday   
1012818  200 block of bridgeview dr  wednesday   

                                    descript  \
40972                          lost property   
41925               checks, forgery (felony)   
203268                         lost property   
245057                    child abuse sexual   
324590                obscene phone calls(s)   
386011   forgery, drivers license or id-card   
415338         fraudulent credit application   
468458       checks, make or pass fictitious   
549495           